<h1> NLP Application </h1>

<h2> by Nathan Dilla & John Haviland </h2>

<h3> Problem Statement </h3>


<h2> Dataset Overview </h2>
<h3> Purpose </h3>




<h3> Step 1: Import Libraries </h3>



In [ ]:
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from nltk.chunk import ChunkParserI
from nltk.tag import UnigramTagger, BigramTagger
from nltk.chunk.util import tree2conlltags, conlltags2tree
from nltk.metrics import precision, recall, f_measure
from sklearn.model_selection import train_test_split

<h3> Step 2: Load in Dataset </h3>



In [ ]:
data = pd.read_csv('hate_speech_dataset.csv')

<h3> Step 3: Download NLTK Data (Optional) </h3>

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

<h3> Step 4: Tokenize, POS Tag Data </h3>

In [ ]:
data['tokens'] = data['OriginalTweet'].apply(word_tokenize)
data['pos_tags'] = data['tokens'].apply(pos_tag)

<h3> Step 5: Split Annotated Data into Training & Test Sets </h3>

In [ ]:
def split_data(data, test_size=0.2, random_state=None):
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    return train_data, test_data

# Define your annotated data in the format (word, ne_tag)
annotated_data = [
    [
        ('Word1', 'B-ENTITY'),
        ('Word2', 'I-ENTITY'),
        ('Word3', 'O'),
        # More words and tags for the first sentence
    ],
    [
        ('Word1', 'B-ENTITY'),
        ('Word2', 'I-ENTITY'),
        ('Word3', 'I-ENTITY'),
        # More words and tags for the second sentence
    ],
    # More sentences
]

# Split your annotated data into training and test sets
train_data, test_data = split_data(annotated_data, test_size=0.2, random_state=42)

<h3> Step 6: Define NER Chunker

In [ ]:
class CustomCoronaNER(ChunkParserI):
    def __init__(self, train_sents):
        # Convert the annotated data to the format used in the example
        train_data = [([(word, ne_tag) for (word, ne_tag) in sent]) for sent in train_sents]
        self.tagger = BigramTagger(train_data, backoff=UnigramTagger(train_data))

    def parse(self, sentence):
        # Predict chunk tags
        tagged_ne_tags = self.tagger.tag(sentence)
        conlltags = [(word, 'POS', ne_tag) for (word, ne_tag) in tagged_ne_tags]
        # Convert to tree
        return conlltags2tree(conlltags)


<h3> Step 7: Train & Evaluate NER System

In [ ]:
corona_ner = CustomCoronaNER(train_data)

gold = [tree2conlltags(conlltags2tree([(word, 'POS', ne_tag) for word, ne_tag in sent])) for sent in test_data]
test = [tree2conlltags(corona_ner.parse(sent)) for sent in test_data]


<h3> Step 8: Compute Performance Metrics </h3>

In [ ]:
gold_chunks = set(gold[0] + gold[1])
test_chunks = set(test[0] + test[1])

precision_score = precision(gold_chunks, test_chunks)
recall_score = recall(gold_chunks, test_chunks)
f_measure_score = f_measure(gold_chunks, test_chunks)

print('Precision:', precision_score)
print('Recall:', recall_score)
print('F-measure:', f_measure_score)

<h3> Analysis of our Findings </h3>



<h3> References </h3>
